<a href="https://colab.research.google.com/github/aryan-cs/poker-like-games/blob/discrete-poker/dominance_and_equilibria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Helper Functions

In [ ]:
def boxed(text):
    lines = text.splitlines()
    if lines and lines[0] == "": lines = lines[1:]
    if not lines: return

    max_width = max(map(len, lines))

    print("┌" + "─" * (max_width + 2) + "┐")
    for line in lines:
      print(f"│ {line.ljust(max_width)} │")
    print("└" + "─" * (max_width + 2) + "┘")

def cutoff_expected_payout_A(kA, kB, n):

  # (-3kAkB + kA² + 2kB² - 2kB + 2kA - nkB + nkA) / (n (n - 1)), {kB > kA}
  if kB > kA:
    return ((-3 * kA * kB )+ (kA * kA) + (2 * kB * kB) - (2 * kB) + (2 * kA) - (n * kB) + (n * kA)) / (n * (n - 1))

  # (-3kAkB + kB² + 2kA² - 2kA + 2kB - nkA + nkB) / (n (n - 1)), {kA > kB}
  elif kA > kB: return ((3 * kA * kB) - (kB * kB) - (2 * kA * kA) + (2 * kA) - (2 * kB) + (n * kA) - (n * kB)) / (n * (n - 1))

  else: return 0

def cutoff_expected_payout_B(kA, kB, n):
  return -1 * cutoff_expected_payout_A(kA, kB, n) # Remember, this is a zero-sum game

# Searching for Dominant Strategies

In [ ]:
#@title { vertical-output: true }

import itertools

n = 3

def indices_of_max(data):
    if not data: return []
    max_value = max(data)
    return [index for index, value in enumerate(data) if value == max_value]

def test_betting_strategies(p1_hands, p2_hands):
    p1_payouts = []
    p1_strategies = []

    for p2_betting_set in p2_hands:
        p1_payouts.clear()
        p1_strategies.clear()

        for p1_betting_set in p1_hands:
            p1_expected_payout = 0
            p2_expected_payout = 0

            for p1_card in range(1, n + 1):
                for p2_card in range(1, n + 1):
                    if p1_card == p2_card:
                        continue

                    if (p1_card in p1_betting_set) and (p2_card in p2_betting_set):
                        if p1_card > p2_card:
                            p1_expected_payout += 2
                            p2_expected_payout -= 2
                        else:
                            p1_expected_payout -= 2
                            p2_expected_payout += 2

                    elif (p1_card not in p1_betting_set) and (p2_card not in p2_betting_set):
                        if p1_card > p2_card:
                            p1_expected_payout += 1
                            p2_expected_payout -= 1
                        else:
                            p1_expected_payout -= 1
                            p2_expected_payout += 1

                    elif (p1_card in p1_betting_set) and (p2_card not in p2_betting_set):
                        p2_expected_payout -= 1
                        p1_expected_payout += 1

                    elif (p1_card not in p1_betting_set) and (p2_card in p2_betting_set):
                        p2_expected_payout += 1
                        p1_expected_payout -= 1

            p1_strategies.append(p1_betting_set)
            p1_payouts.append(p1_expected_payout)

        p1_maxes = indices_of_max(p1_payouts)
        bad_strategies = []
        bad_payouts = 2
        is_possible = False

        for index in p1_maxes:
            strat = list(p1_strategies[index])
            works = all(strat[len(strat) - i - 1] == n - i for i in range(len(strat)))

            if works:
                is_possible = True
            else:
                bad_strategies = p1_strategies[index]
                bad_payouts = p1_payouts[index]

        # Print the optimal strategy for Player A given Player B's strategy
        # boxed(
        #     "P1 Strat: " + str(p1_strategies[index]) + "\n" +
        #     "Payout: " + str(p1_payouts[index])
        # )

        if not is_possible:
            boxed(
              "P2 Strat: " + str(p2_betting_set) + "\n"
              "Payout: " + str(bad_payouts) + "\n"
              "Not Cutoff P1 Strat: " + str(bad_strategies) + "\n"
            )

# Example usage
p1_hands = [[], [1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]
p2_hands = [[], [1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]

test_betting_strategies(p1_hands, p2_hands)

┌─────────────────────────────┐
│ P2 Strat: [3]               │
│ Payout: 1                   │
│ Not Cutoff P1 Strat: [1, 3] │
└─────────────────────────────┘
